In [8]:
%pip install tensorflow opencv-python mediapipe scikit-learn matplotlib

  Using cached mediapipe-0.10.14-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached cffi-1.17.0-cp312-cp312-win_amd64.whl.metadata (1.6 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ---------------------------------------- 0.0/50.8 MB ? eta -:--:--
   --- ------------------------------------ 4.2/50.8 MB 19.4 MB/s eta 0:00:03
   ------- -------------------------------- 8.9/50.8 MB 21.3 MB/s eta 0:00:02
   -------- ------------------------------- 10.5/50.8 MB 21.1 MB/s eta 0:00:02
   -------- ------------------------------- 10.5/50.8 MB 21.1 MB/s eta 0:00:02
   -------- ------------------------------- 10.5/50.8 MB 21.1 MB/s eta 0:00:02
   ---------- ----------------------------- 12.8/50.8 MB 10.2 MB/s eta 0:00:04
   ------------- -------------------------- 17.6/50.8 MB 11.9 MB/s eta 0:00:03
   ----------------- ---------------------- 22.5/50.8 MB 13.5 MB/s eta 0:00:03
   -------------------

In [12]:
import os 
from matplotlib import pyplot as plt
import time
import numpy as np
import mediapipe as mp
import cv2

In [13]:
mph=mp.solutions.holistic
mpd=mp.solutions.drawing_utils

In [14]:
def detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable=False
    results=model.process(image)
    image.flags.writeable=True
    image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results

In [17]:
capture = cv2.VideoCapture(0)
with mph.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as model:
    while capture.isOpened():
        ret, frame = capture.read()
        image,results=detection(frame,model)
        cv2.imshow('Sign Language Recognition',frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    capture.release()
    cv2.destroyAllWindows()